# Research-Oriented Operating Systems: Chapter 2 - Process Concepts

This Jupyter Notebook explores the core concepts of processes in operating systems, focusing on **Process States**, **Process Control Block (PCB)**, and **Context Switching**. It provides theoretical insights, practical simulations, visualizations, research directions, and projects tailored for researchers and scientists.

## Objectives
- Understand process states, PCB structure, and context switching mechanics.
- Simulate process management using Python.
- Explore research trends and applications in process management.
- Develop hands-on skills through tutorials and projects.

## Prerequisites
- Python 3 with `pandas`, `matplotlib`, `numpy`.
- VirtualBox and Ubuntu ISO for lab setup.
- Basic knowledge of OS concepts (*Operating System Concepts*, Chapter 3 recommended).

## Current Date and Time
- 11:22 AM IST, Wednesday, July 30, 2025


## 1. Process States

### Theory
A process is a program in execution, transitioning through states:
- **New**: Process is being created.
- **Ready**: Process is waiting for CPU allocation.
- **Running**: Process is executing on the CPU.
- **Waiting (Blocked)**: Process is waiting for an event (e.g., I/O).
- **Terminated**: Process has completed execution.

**Rare Insight**: In real-time systems, additional states (e.g., Suspended) are used to manage priority inversion, critical for predictable scheduling.

**Applications**:
- **Multitasking**: Process states enable concurrent execution in desktops and servers.
- **Embedded Systems**: Lightweight state management for resource-constrained devices.

### Practical Code: Simulate Process State Transitions
Simulate a process transitioning through states based on events.


In [ ]:
class Process:
    def __init__(self, pid):
        self.pid = pid
        self.state = 'New'
        self.transitions = []

    def transition(self, event):
        if self.state == 'New' and event == 'admit':
            self.state = 'Ready'
        elif self.state == 'Ready' and event == 'schedule':
            self.state = 'Running'
        elif self.state == 'Running' and event == 'io_request':
            self.state = 'Waiting'
        elif self.state == 'Waiting' and event == 'io_complete':
            self.state = 'Ready'
        elif self.state == 'Running' and event == 'complete':
            self.state = 'Terminated'
        self.transitions.append((event, self.state))

    def get_state(self):
        return self.state

# Simulate process
proc = Process(1)
events = ['admit', 'schedule', 'io_request', 'io_complete', 'schedule', 'complete']
for event in events:
    proc.transition(event)
    print(f'PID: {proc.pid}, Event: {event}, State: {proc.get_state()}')


### Visualization: State Transitions
Visualize the frequency of each state during the simulation.


In [ ]:
# Visualization using matplotlib (fix: replace invalid chartjs block with working Python code)
import matplotlib.pyplot as plt
from collections import Counter

# Re-run the simulation to collect state transitions
proc = Process(1)
events = ['admit', 'schedule', 'io_request', 'io_complete', 'schedule', 'complete']
states = ['New']
for event in events:
    proc.transition(event)
    states.append(proc.get_state())

state_counts = Counter(states)
all_states = ['New', 'Ready', 'Running', 'Waiting', 'Terminated']
frequencies = [state_counts.get(state, 0) for state in all_states]

plt.figure(figsize=(7,4))
plt.bar(all_states, frequencies, color=['#36A2EB', '#FF6384', '#FFCE56', '#4BC0C0', '#9966FF'])
plt.xlabel('Process State')
plt.ylabel('Frequency')
plt.title('Process State Transitions')
plt.show()


### Research Direction
- **Lightweight Processes**: Investigate user-level threads (e.g., in unikernels) for reduced state transition overhead.
- **Real-Time Scheduling**: Explore state management for deterministic execution in RTOS.


## 2. Process Control Block (PCB)

### Theory
The PCB is a data structure that stores all information about a process, including:
- **Process ID (PID)**: Unique identifier.
- **State**: Current state (e.g., Ready, Running).
- **Program Counter**: Address of the next instruction.
- **Registers**: CPU register values.
- **Memory Info**: Memory allocation details.
- **Priority**: Scheduling priority.

**Rare Insight**: PCB size impacts performance; modern OS optimize PCB storage using cache-friendly designs or compressed data structures.

**Applications**:
- **Cloud Computing**: PCBs enable efficient VM process management.
- **Real-Time Systems**: PCBs store priority and deadline information for predictable scheduling.

### Practical Code: Simulate PCB Management
Simulate creating and updating PCBs for multiple processes.


In [ ]:
import pandas as pd

class PCB:
    def __init__(self, pid, state='New', pc=0, priority=0):
        self.pcb = {
            'PID': pid,
            'State': state,
            'ProgramCounter': pc,
            'Priority': priority,
            'Registers': {'R1': 0, 'R2': 0},
            'Memory': '0x1000'
        }

    def update_state(self, state):
        self.pcb['State'] = state

    def update_pc(self, pc):
        self.pcb['ProgramCounter'] = pc

    def get_pcb(self):
        return self.pcb

    def get_state(self):
        return self.pcb['State']

# Simulate PCB management
processes = [PCB(pid=i, priority=i) for i in range(3)]
processes[0].update_state('Running')
processes[0].update_pc(100)
processes[1].update_state('Ready')
processes[2].update_state('Waiting')

# Display PCBs
pcb_data = [p.get_pcb() for p in processes]
print(pd.DataFrame(pcb_data))


### Research Direction
- **PCB Optimization**: Explore compact PCB designs for large-scale systems (e.g., cloud servers).
- **Security**: Investigate PCB tampering prevention in secure OS (e.g., seL4).


## 3. Context Switching

### Theory
Context switching is the process of saving and restoring a process’s state (via PCB) to switch CPU execution between processes. Steps include:
- Save current process’s registers and PCB.
- Load new process’s PCB and registers.
- Update scheduler state.

**Rare Insight**: Context switching overhead in virtualized environments (e.g., VMs, containers) is amplified due to hypervisor involvement, a key challenge in cloud computing.

**Applications**:
- **Multitasking**: Enables seamless process switching in desktops.
- **Real-Time Systems**: Minimizing context switch latency is critical for deadlines.

### Practical Code: Simulate Context Switching
Simulate context switching with overhead measurement.


In [ ]:
import time
import numpy as np

class ContextSwitchSimulator:
    def __init__(self):
        self.current_pid = None
        self.overhead = []

    def context_switch(self, new_pid):
        start_time = time.time()
        # Simulate saving and loading PCB (1ms overhead)
        time.sleep(0.001)
        self.current_pid = new_pid
        self.overhead.append((time.time() - start_time) * 1000)  # ms

    def get_overhead(self):
        return np.mean(self.overhead) if self.overhead else 0.0

# Simulate context switches
sim = ContextSwitchSimulator()
for pid in [1, 2, 3, 1, 2]:
    sim.context_switch(pid)
    print(f'Context Switch to PID: {pid}')
print(f'Average Context Switch Overhead: {sim.get_overhead():.3f} ms')


### Visualization: Context Switch Overhead
Visualize the overhead of context switches.


In [ ]:
# Visualization using matplotlib (fix: replace invalid chartjs block with working Python code)
import matplotlib.pyplot as plt

# Simulate and collect overheads
sim = ContextSwitchSimulator()
pids = [1, 2, 3, 1, 2]
overheads = []
for pid in pids:
    sim.context_switch(pid)
    overheads.append(sim.overhead[-1])

plt.figure(figsize=(7,4))
plt.plot(range(1, len(overheads)+1), overheads, marker='o', color='#36A2EB', label='Context Switch Overhead (ms)')
plt.xlabel('Switch Number')
plt.ylabel('Overhead (ms)')
plt.title('Context Switch Overhead')
plt.legend()
plt.grid(True)
plt.show()


### Research Direction
- **Context Switch Optimization**: Investigate hardware-assisted context switching (e.g., Intel VT-x).
- **Virtualization**: Reduce context switch overhead in hypervisors for cloud systems.


## Tutorial: Exploring Processes in Ubuntu

**Objective**: Use Ubuntu in VirtualBox to explore process management.

**Steps**:
1. Install VirtualBox (https://www.virtualbox.org/) and Ubuntu (https://ubuntu.com/download).
2. Create a VM (2GB RAM, 20GB storage, 2 CPUs).
3. Install Ubuntu and open a terminal.
4. Run:
   ```bash
   ps -aux  # List all processes
   top      # Monitor processes
   pstree   # View process hierarchy
   ```

**Code**: Simulate `ps` output.


In [ ]:
import pandas as pd

# Mock process data
processes = [
    {'PID': 1, 'USER': 'root', 'STATE': 'Running', 'COMMAND': 'init', '%CPU': 0.1},
    {'PID': 123, 'USER': 'user', 'STATE': 'Waiting', 'COMMAND': 'python', '%CPU': 1.5},
    {'PID': 456, 'USER': 'user', 'STATE': 'Ready', 'COMMAND': 'firefox', '%CPU': 3.2}
]

print(pd.DataFrame(processes))


## Mini Project: Process Scheduler with PCB

**Objective**: Simulate a Round-Robin scheduler with PCB updates and state transitions.

**Code**:


In [ ]:
import numpy as np

class Scheduler:
    def __init__(self, quantum):
        self.quantum = quantum
        self.processes = []
        self.time = 0

    def add_process(self, pid, burst_time):
        pcb = PCB(pid, state='Ready', pc=0, priority=0)
        self.processes.append({'PCB': pcb, 'Burst': burst_time, 'Remaining': burst_time})

    def run(self):
        while any(p['Remaining'] > 0 for p in self.processes):
            for proc in self.processes:
                if proc['Remaining'] > 0:
                    proc['PCB'].update_state('Running')
                    run_time = min(self.quantum, proc['Remaining'])
                    proc['Remaining'] -= run_time
                    proc['PCB'].update_pc(proc['PCB'].get_pcb()['ProgramCounter'] + run_time * 10)
                    self.time += run_time
                    proc['PCB'].update_state('Ready' if proc['Remaining'] > 0 else 'Terminated')
                    print(f'PID: {proc["PCB"].get_pcb()["PID"]}, State: {proc["PCB"].get_state()}, PC: {proc["PCB"].get_pcb()["ProgramCounter"]}')

# Simulate
scheduler = Scheduler(quantum=2)
scheduler.add_process(1, 5)
scheduler.add_process(2, 3)
scheduler.run()


## Major Project: Process Management System with Context Switching

**Objective**: Design a system to manage processes, PCBs, and context switching.

**Outline**:
- **Process Manager**: Maintains PCBs and state transitions.
- **Context Switcher**: Simulates saving/loading PCBs.
- **Scheduler**: Round-Robin with context switch overhead.

**Code**:


In [ ]:
import time
import numpy as np

class ProcessManager:
    def __init__(self, quantum):
        self.quantum = quantum
        self.processes = []
        self.overhead = []
        self.time = 0

    def add_process(self, pid, burst_time):
        pcb = PCB(pid, state='Ready', pc=0, priority=0)
        self.processes.append({'PCB': pcb, 'Burst': burst_time, 'Remaining': burst_time})

    def context_switch(self, old_pid, new_pid):
        start_time = time.time()
        time.sleep(0.001)  # Simulate overhead
        if old_pid is not None:
            for p in self.processes:
                if p['PCB'].get_pcb()['PID'] == old_pid:
                    p['PCB'].update_state('Ready')
        for p in self.processes:
            if p['PCB'].get_pcb()['PID'] == new_pid:
                p['PCB'].update_state('Running')
        self.overhead.append((time.time() - start_time) * 1000)

    def run(self):
        current_pid = None
        while any(p['Remaining'] > 0 for p in self.processes):
            for proc in self.processes:
                if proc['Remaining'] > 0:
                    self.context_switch(current_pid, proc['PCB'].get_pcb()['PID'])
                    current_pid = proc['PCB'].get_pcb()['PID']
                    run_time = min(self.quantum, proc['Remaining'])
                    proc['Remaining'] -= run_time
                    proc['PCB'].update_pc(proc['PCB'].get_pcb()['ProgramCounter'] + run_time * 10)
                    self.time += run_time
                    if proc['Remaining'] == 0:
                        proc['PCB'].update_state('Terminated')
                    print(f'PID: {current_pid}, State: {proc["PCB"].get_state()}, PC: {proc["PCB"].get_pcb()["ProgramCounter"]}')

# Simulate
pm = ProcessManager(quantum=2)
pm.add_process(1, 5)
pm.add_process(2, 3)
pm.run()
print(f'Average Context Switch Overhead: {np.mean(pm.overhead):.3f} ms')


## Additional Content for Scientists

### Security Considerations
- **PCB Security**: Protect PCBs from unauthorized access (e.g., kernel exploits).
- **Context Switching**: Ensure secure state restoration to prevent data leaks.

### Performance Optimization
- **Context Switch Reduction**: Use thread affinity to minimize switches.
- **PCB Caching**: Optimize PCB storage for cache locality.

### Advanced Formal Modeling
- **State Machines**: Model process state transitions for verification.
- **Process Algebras**: Use CSP to analyze deadlock-free scheduling.

**Code**: Conceptual CSP model for process interactions.


In [ ]:
class CSPModel:
    def __init__(self):
        self.events = []

    def process(self, name, event):
        self.events.append(f'{name} -> {event}')

# Simulate
csp = CSPModel()
csp.process('P1', 'schedule')
csp.process('P2', 'io_request')
print(csp.events)


## Conclusion

This notebook provides a comprehensive exploration of process concepts, including states, PCBs, and context switching. Key takeaways:
- Process states enable multitasking and resource sharing.
- PCBs store critical process metadata for efficient management.
- Context switching is essential but introduces performance overhead.

**Next Steps**:
- Extend the scheduler to support priority-based scheduling.
- Implement a real-time process manager in C/Rust.
- Use formal verification tools (e.g., Isabelle/HOL) to prove scheduler correctness.
